In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

import datetime

from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

In [30]:
#load data
data = pd.read_csv('data/milk.csv')
print data.columns

#include incumbent to indentify things

Index([u'VENDOR', u'WW', u'WC', u'LFW', u'LFC', u'WIN', u'SYSTEM', u'YEAR',
       u'MONTH', u'DAY', u'FMOZONE', u'ESC', u'COOLER', u'QLFC', u'QLFW',
       u'QWW', u'QWC', u'ESTQTY', u'DEL', u'MILES', u'NUMSCHL', u'NUMWIN',
       u'POPUL', u'ADJPOP', u'NUM', u'GAS', u'FMO'],
      dtype='object')


In [22]:
data_dates = data.loc[ (data['DAY']>0) & (data['MONTH']>0)]
dates = pd.to_datetime(data_dates[['YEAR','MONTH','DAY']])
data_dates.loc[:,'DATE'] = dates

In [24]:
reg1 = data_dates[['WW','FMO','DATE']].dropna()
reg1 = reg1[reg1['WW']<.4]

reg1[['WW','FMO']] = np.log(reg1[['WW','FMO']])
model = sm.OLS(reg1['WW'],sm.add_constant(reg1['FMO']))
result = model.fit()
print result.summary()

                            OLS Regression Results                            
Dep. Variable:                     WW   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     138.9
Date:                Wed, 24 Apr 2019   Prob (F-statistic):           1.60e-31
Time:                        15:02:36   Log-Likelihood:                 3793.9
No. Observations:                3925   AIC:                            -7584.
Df Residuals:                    3923   BIC:                            -7571.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6225      0.071    -37.034      0.0

In [29]:
model = MarkovRegression(endog=reg1['WW'], exog=sm.add_constant(reg1['FMO']),
                         k_regimes=2, order=0, trend='nc',switching_exog=True)
result = model.fit()
print result.summary()

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/tsa/base/tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                     WW   No. Observations:                 3925
Model:               MarkovRegression   Log Likelihood                3793.929
Date:                Wed, 24 Apr 2019   AIC                          -7573.858
Time:                        15:04:37   BIC                          -7529.932
Sample:                             0   HQIC                         -7558.273
                               - 3925                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2441        nan        nan        nan         nan         nan
FMO            0.0023        nan        nan        n

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/tsa/regime_switching/markov_switching.py:1782: RuntimeWarning: divide by zero encountered in true_divide
  return 1. / (1 - np.diagonal(self.regime_transition).squeeze())
/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [ ]:
#ESC, COOLER ,MILES, GAS, DEL, ESTQTY
reg1 = data[['WW', 'FMO','ESC', 'NUMSCHL','MILES','COOLER', 'GAS','DEL','ESTQTY']].dropna()

#make a little better
reg1['NUMSCHL'] = np.maximum(reg1['NUMSCHL'],1)
reg1['NOSTOP'] = reg1['NUMSCHL']*reg1['DEL']
reg1['MILES'] =reg1['MILES']+1
reg1 = reg1[reg1['WW']<=.3]

#log scale
reg1[['WW', 'FMO', 'MILES', 'GAS','NOSTOP', 'ESTQTY']] = np.log(
                                            reg1[['WW', 'FMO', 'MILES', 'GAS', 'NOSTOP', 'ESTQTY']])

print reg1.mean()
